In [1]:
# Libraries
import open3d as o3d
import viser
import torch
import numpy as np
import cv2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#@title Importing the Clusters with It's radiances
slices = torch.load("/workspace/FruitProposal/attachment/RadianceCloud/slice_points.pt")
print(slices.keys())

planes  = slices['plane']
points  = slices['points']
rgbs    = slices['rgb']
centers = slices['slice_center']
corners = slices['slice_corner']

print(planes.shape, points.shape, rgbs.shape, centers.shape, corners.shape)

dict_keys(['points', 'plane', 'rgb', 'slice_corner', 'slice_center'])
(2733, 3) (2733, 3) (2733, 3) (9, 3) (9, 4, 3)


In [ ]:
# planes, points, centers and corners preprocessing
_planes = planes.copy()
_centers = centers.copy()
_corners = []

# To remplace z=n to z=1 all cases planes, centers and corners
_planes[:, 2] = 1.0
_centers[:, 2] = 1.0


_displayed_corners = []
for c in _corners:
    c[:,2] = 1.0
    _corners.append(c)
    for i in c:
        _displayed_corners.append(i)

_corners = np.array(_corners)
_displayed_corners = np.array(_displayed_corners)
print(_displayed_corners)


# Converting to Open3D PointCloud
pcd_planes = o3d.geometry.PointCloud()
pcd_planes.points = o3d.utility.Vector3dVector(_planes)
pcd_planes.colors = o3d.utility.Vector3dVector(rgbs)

pcd_corners = o3d.geometry.PointCloud()
pcd_corners.points = o3d.utility.Vector3dVector(_displayed_corners)
pcd_corners.colors = o3d.utility.Vector3dVector(np.ones_like(_displayed_corners) * [1, 0, 0])  # Green color for corners

pcd_centers = o3d.geometry.PointCloud()
pcd_centers.points = o3d.utility.Vector3dVector(_centers)
pcd_centers.colors = o3d.utility.Vector3dVector(np.ones_like(_centers) * [0, 1, 0])  # Red color for centers

# Initializing the visualizer server
server = viser.ViserServer()

# Adding the point cloud to the server
server.scene.add_point_cloud(
    name="/planes",
    points=np.asarray(pcd_planes.points),
    colors=np.asarray(pcd_planes.colors),
    point_size=0.0005,
    visible=True
    )

server.scene.add_point_cloud(
    name="/corners",
    points=np.asarray(pcd_corners.points),
    colors=np.asarray(pcd_corners.colors),
    point_size=0.01,
    visible=True
)

server.scene.add_point_cloud(
    name="/centers",
    points=np.asarray(pcd_centers.points),
    colors=np.asarray(pcd_centers.colors),
    point_size=0.01,
    visible=True
)

# while True:
#     x = input("Press Enter to continue...")
#     if x == "":
#         break



[[0.24521568 0.18485084 1.        ]
 [0.2971612  0.18485084 1.        ]
 [0.24521568 0.23679635 1.        ]
 [0.2971612  0.23679635 1.        ]
 [0.24521568 0.18485084 1.        ]
 [0.2971612  0.18485084 1.        ]
 [0.24521568 0.23679635 1.        ]
 [0.2971612  0.23679635 1.        ]
 [0.24521568 0.18485084 1.        ]
 [0.2971612  0.18485084 1.        ]
 [0.24521568 0.23679635 1.        ]
 [0.2971612  0.23679635 1.        ]
 [0.24521568 0.18485084 1.        ]
 [0.2971612  0.18485084 1.        ]
 [0.24521568 0.23679635 1.        ]
 [0.2971612  0.23679635 1.        ]
 [0.24521568 0.18485084 1.        ]
 [0.2971612  0.18485084 1.        ]
 [0.24521568 0.23679635 1.        ]
 [0.2971612  0.23679635 1.        ]
 [0.24521568 0.18485084 1.        ]
 [0.2971612  0.18485084 1.        ]
 [0.24521568 0.23679635 1.        ]
 [0.2971612  0.23679635 1.        ]
 [0.24521568 0.18485084 1.        ]
 [0.2971612  0.18485084 1.        ]
 [0.24521568 0.23679635 1.        ]
 [0.2971612  0.23679635 1.  

╭─────────────── viser ───────────────╮
│             ╷                       │
│   HTTP      │ http://0.0.0.0:8080   │
│   Websocket │ ws://0.0.0.0:8080     │
│             ╵                       │
╰─────────────────────────────────────╯

PointCloudHandle(_impl=_SceneNodeHandleState(name='/centers', api=<viser._scene_api.SceneApi object at 0x7f92ab5e1360>, wxyz=array([1., 0., 0., 0.]), position=array([0., 0., 0.]), visible=True, click_cb=None))

In [ ]:
# Precompute intrinsics once
z_cam = np.array([0, 0, -1])
W = H = 512
cx = W / 2
cy = H / 2
fxy = W
intrinsic = np.array([
    [fxy,   0,  cx],
    [  0, fxy,  cy],
    [  0,   0,   1]
])

e       = _corners[0][0] - _corners[0][1]
e_norm  = np.linalg.norm(e)
y_cam   = e / e_norm
x_cam   = np.cross(y_cam, z_cam);  x_cam /= np.linalg.norm(x_cam)
y_cam   = np.cross(z_cam, x_cam);  y_cam /= np.linalg.norm(y_cam)
R       = np.column_stack((x_cam, y_cam, z_cam))

for k, (pts, plane, rgb, corners_k, center_k) in enumerate(
        zip(points, planes, rgbs, corners, centers, strict=False)):

    # 1) elevar la slice por encima de Z=0
    z0      = plane[:, 2].min()
    shift_z = max(0.05 - z0, 0.0)

    # 3) posición inicial de la cámara
    center_shift = center_k.copy()
    center_shift[2] += shift_z
    Ccam = center_shift + np.array([0, 0, e_norm])

    # --- ajuste fino: centrar la proyección del centro del slice ---
    Xc_center = R @ (center_shift - Ccam)
    uvw_c     = intrinsic @ Xc_center
    u_c, v_c  = uvw_c[0]/uvw_c[2], uvw_c[1]/uvw_c[2]
    delta_u, delta_v = cx - u_c, cy - v_c

    # pasar de píxeles a desplazamiento en el espacio de cámara
    # un píxel en x corresponde a 1/fxy unidades en x_cam, igual para y
    Ccam = Ccam - (delta_u / fxy) * x_cam - (delta_v / fxy) * y_cam

    # 4) preparar lienzo
    void_image = np.zeros((H, W, 3), dtype=np.uint8)

    # 5) proyectar cada punto
    for pt, col in zip(plane, rgb):
        p = pt.copy();  p[2] += shift_z
        Xw_cam0 = p - Ccam
        Xc      = R @ Xw_cam0
        if Xc[2] <= 0:
            continue

        uvw = intrinsic @ Xc
        u, v = uvw[0]/uvw[2], uvw[1]/uvw[2]
        ui, vi = int(round(u)), int(round(v))
        if 0 <= ui < W and 0 <= vi < H:
            void_image[vi, ui] = (np.array(col)*255).astype(np.uint8)[::-1]

    # 6) dilatar y guardar
    kernel = np.ones((3,3), np.uint8)
    out = cv2.dilate(void_image, kernel, iterations=5)
    cv2.imwrite(f"/workspace/FruitProposal/attachment/RadianceCloud/images/slice_{k}.png", out)


[[512.   0. 256.]
 [  0. 512. 256.]
 [  0.   0.   1.]]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

(viser) Connection opened (0, 1 total), 16 persistent messages

(viser) Connection closed (0, 0 total)

(viser) Connection opened (1, 1 total), 16 persistent messages